
"We believe that for the recent past and foreseeable future, off-chip memory bandwidth will often be the constraining resource"'


In [ ]:
library(ggplot2)


intensity = 2^(-8:4)

offchip = 16 # GB/s
flopsmax = 16 # GFLOPs
        
memband <- data.frame(intensity, intensity * 16, flopsmax )
colnames(memband) <- c("intensity", "FLOPS", 'cpuFLOPS')

 
       
aplot <- ggplot(memband, aes(x=intensity, y=FLOPS, color="off-chip")) + geom_line() +
            geom_line(data=memband, aes(x=intensity, y=cpuFLOPS, color="cpu")) +
            coord_trans(x="log2", y="log2") + scale_y_continuous(limits=c(1,50))

print(aplot)
